In [1]:
import transformers
import torch
transformers.BERT_PRETRAINED_MODEL_ARCHIVE_MAP["bert-base-finnish-cased"]="http://dl.turkunlp.org/finbert/torch-transformers/bert-base-finnish-cased/pytorch_model.bin"
transformers.BERT_PRETRAINED_CONFIG_ARCHIVE_MAP["bert-base-finnish-cased"]="http://dl.turkunlp.org/finbert/torch-transformers/bert-base-finnish-cased/config.json"
transformers.tokenization_bert.PRETRAINED_VOCAB_FILES_MAP["vocab_file"]["bert-base-finnish-cased"]="http://dl.turkunlp.org/finbert/torch-transformers/bert-base-finnish-cased/vocab.txt"
transformers.tokenization_bert.PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES["bert-base-finnish-cased"]=512
transformers.tokenization_bert.PRETRAINED_INIT_CONFIGURATION["bert-base-finnish-cased"]={'do_lower_case': False}

print("CUDA:",torch.cuda.is_available())

To use data.metrics please install scikit-learn. See https://scikit-learn.org/stable/index.html


CUDA: True


In [2]:
import gen_finetune_dataset as ds

In [3]:
model=transformers.BertForMaskedLM.from_pretrained("bert-base-finnish-cased")
tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-finnish-cased")

In [4]:
model=model.cuda()

In [5]:
import gzip
f=open("xxx.txt","rt")
doc_examples=ds.doc_examples_from_plaintext(f)
batches=ds.batches_from_documents(doc_examples,tokenizer)



In [6]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
optimizer=transformers.optimization.AdamW(optimizer_grouped_parameters,lr=0.001)
scheduler = transformers.WarmupLinearSchedule(optimizer, warmup_steps=100, t_total=10000)
model.zero_grad()
model.train()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50105, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [7]:
for idx,x in enumerate(batches):
    inp,mask,outp=x
    inp=inp.cuda()
    mask=mask.cuda()
    outp=outp.cuda()
    optimizer.zero_grad()
    outputs=model(inp,attention_mask=mask,masked_lm_labels=outp)
    loss=outputs[0]
    loss.backward()
    optimizer.step()
    scheduler.step()
    print("%.05f     %d"%(loss.item(),idx),end="\r")
    
    
    
    
    

KeyboardInterrupt: 

In [ ]:
torch.cuda.empty_cache()
print("MEM=",torch.cuda.get_device_properties(0).total_memory)

In [ ]:
x=torch.ones((5,),dtype=torch.long)
x[1:2]


In [ ]:
loss.item()